In [20]:
import os
import pandas as pd
import sqlite3


In [21]:
#region VERİTABANI OLUŞTURMA VE VERİ AKTARMA
current_dir = os.getcwd()
file_path = os.path.join(current_dir, 'worldcities.csv')
df = pd.read_csv(#dataframe'i oku
    file_path
)
#sayılsal kolonları dönüştür 
df["population"] =pd.to_numeric(df["population"], errors='coerce')
#sqlite veritabanına bağlan
connection = sqlite3.connect("worldcities.db")
#dataframii tabloya aktar
df.to_sql("cities", connection, if_exists="replace", index=False)
#endregion  

48059

In [22]:
queries = {
    "en_kalabalik_10":
    """
        SELECT city, country, population 
        FROM cities 
        ORDER BY population DESC 
        LIMIT 10;
    """,
    "turkiye_sehirleri":
    """
        SELECT city, population 
        FROM cities 
        WHERE country = 'Turkey' 
        ORDER BY population DESC
        limit 10;
    """,
    "ulke_bazli_nufus":
    """
        SELECT country, SUM(population) as total_pop
        FROM cities 
        GROUP BY country 
        ORDER BY total_pop DESC 
        LIMIT 5;
    """
}

seçilen sorguyu çalıştır

In [23]:
selected_query = "en_kalabalik_10"#örnek olarak bir sorgu seç
print(pd.read_sql_query(queries[selected_query],connection))

          city       country  population
0        Tokyo         Japan  37785000.0
1      Jakarta     Indonesia  33756000.0
2        Delhi         India  32226000.0
3    Guangzhou         China  26940000.0
4       Mumbai         India  24973000.0
5       Manila   Philippines  24922000.0
6     Shanghai         China  24073000.0
7    São Paulo        Brazil  23086000.0
8        Seoul  Korea, South  23016000.0
9  Mexico City        Mexico  21804000.0


her sorguyu çalıştır ve sonucu yazdır

In [24]:
for item in queries:
    print(f"\nSorgu: {item}")
    print(pd.read_sql_query(queries[item], connection))
print("Sorgular tamamlandı.")


Sorgu: en_kalabalik_10
          city       country  population
0        Tokyo         Japan  37785000.0
1      Jakarta     Indonesia  33756000.0
2        Delhi         India  32226000.0
3    Guangzhou         China  26940000.0
4       Mumbai         India  24973000.0
5       Manila   Philippines  24922000.0
6     Shanghai         China  24073000.0
7    São Paulo        Brazil  23086000.0
8        Seoul  Korea, South  23016000.0
9  Mexico City        Mexico  21804000.0

Sorgu: turkiye_sehirleri
         city  population
0    Istanbul  14441000.0
1      Ankara   5864049.0
2       Bursa   3101833.0
3       İzmir   2965900.0
4       Konya   2320241.0
5   Gaziantep   2130432.0
6  Diyarbakır   1791373.0
7       Adana   1765981.0
8     Kayseri   1434357.0
9      Samsun   1356079.0

Sorgu: ulke_bazli_nufus
         country     total_pop
0          China  1.360846e+09
1          India  5.146940e+08
2  United States  3.794267e+08
3         Brazil  1.911845e+08
4          Japan  1.848941e+08
So

SORGULARI YAZABİLECEĞİMİZ BİR SORGU FONKSİYONU YAZALIM


In [25]:
# yardımcı fonksiyon queryleri alıp çalıştırır
def run_query(query):
    try:
        result = pd.read_sql_query(query, connection)
        print(result)
    except Exception as e:
        print(f"Sorgu çalıştırılırken hata oluştu: {e}")
        return None

SORGULAR

In [26]:
#popülasyonu 1 milyondan fazla olan şehirleri al ilk 5 tanesini göster
MY_QUERY="""
select city , country
from cities
where population > 1000000
limit 5;
"""
df_query_result = run_query(MY_QUERY)

if df_query_result is not None:
    print(df_query_result)
    

        city    country
0      Tokyo      Japan
1    Jakarta  Indonesia
2      Delhi      India
3  Guangzhou      China
4     Mumbai      India


In [27]:
#Türkiye'de nüfusu 500.000'den fazla olan en kalabalık 6 şehri al
run_query("""
SELECT city, country, population
          from cities
          where country = 'Turkey' AND population > 500000
          order by population DESC
          limit 6;
"""
)

        city country  population
0   Istanbul  Turkey  14441000.0
1     Ankara  Turkey   5864049.0
2      Bursa  Turkey   3101833.0
3      İzmir  Turkey   2965900.0
4      Konya  Turkey   2320241.0
5  Gaziantep  Turkey   2130432.0


In [28]:
#almanyanın en az nüfuslu 3 şehri
run_query("""select city , population
          from cities
          where country ='Germany'
          order by population asc
          limit 3;
          """)

            city  population
0         Erfurt         NaN
1      Ahrensbök      8502.0
2  Bad Feilnbach      8512.0


In [29]:
#almanyanın en kalabalık 4 şehri
run_query("""
        select city , population
          from cities
          where country='Germany'
          order by population desc
          limit 4;
""")

        city  population
0     Berlin   4679500.0
1  Stuttgart   2787724.0
2     Munich   2606021.0
3    Hamburg   2496600.0


In [30]:
run_query("""
select country , sum(population) as total_population
          from cities
          group by country
          order by total_population desc
          limit 5;
""")

         country  total_population
0          China      1.360846e+09
1          India      5.146940e+08
2  United States      3.794267e+08
3         Brazil      1.911845e+08
4          Japan      1.848941e+08


In [31]:
#nüfusu 1000 ile 2000 arasında olan şehirlerden en düşük nüfuslu ilk 7 tanesini al   
run_query("""select country, city, population
          from cities
          where population between 1000 and 2000 
          order by population asc
          limit 7;
          """)

             country         city  population
0             Russia  Chernogorsk      1000.0
1              Sudan         Haya      1000.0
2          Australia    Hughenden      1004.0
3           Slovenia    Mirna Peč      1015.0
4           Slovenia  Gornji Grad      1016.0
5          Australia        Ouyen      1022.0
6  Wallis and Futuna     Mata-Utu      1029.0


In [32]:
#a ile başlayıp a ile biten ülkeler
run_query("select country from cities where country like 'A%' and country like '%A' group by country;")

                country
0               Albania
1               Algeria
2        American Samoa
3               Andorra
4                Angola
5              Anguilla
6   Antigua and Barbuda
7             Argentina
8               Armenia
9                 Aruba
10            Australia
11              Austria


In [ ]:
#kayıtlı şehir sayısı
run_query("select count(id) as total_cities from cities;")

#kayıtlı şehirlerin ortalama nüfusu nedir 
run_query("select avg(population) as avarage_population from cities;")

#tokyo şehrinin nüfusu
run_query("select population as populasyon from cities where city = 'Tokyo' ;")

#en yüksek rakımlı şehir
run_query("select country, city, lat as latiutde from cities order by lat desc limit 1 ;")

#en düşük rakımlı şehrin nüfusu
run_query("select country,city,lat as latitude")

   total_cities
0         48059
   avarage_population
0       107856.626381
   populasyon
0  37785000.0
     country  city  latiutde
0  Greenland  Nord   81.7166


bağlantıyı kapat

In [229]:
connection.close()